In [1]:
from bokeh.plotting import figure, show, output_notebook

In [2]:
output_notebook()

Loading BokehJS ...

In [3]:
import pandas as pd
import numpy as np

In [4]:
import os

In [5]:
DISEASE_NAME = 'HS_iCDf'
DATASET_FOLDER = os.path.join('..', 'datasets/ibd_dataset', DISEASE_NAME)

In [6]:
training_file = os.path.join(DATASET_FOLDER, 'Sokol_16S_taxa_HS_iCDf_commsamp_training.txt')
training_labs = os.path.join(DATASET_FOLDER, 'Sokol_16S_taxa_HS_iCDf_commsamp_training_lab.txt')

test_file = os.path.join(DATASET_FOLDER, 'Sokol_16S_taxa_HS_iCDf_commsamp_test.txt')
test_labs = os.path.join(DATASET_FOLDER, 'Sokol_16S_taxa_HS_iCDf_commsamp_test_lab.txt')

In [7]:
print(os.path.exists(training_file), os.path.exists(training_labs), os.path.exists(test_file), 
os.path.exists(test_labs))

True True True True


In [8]:
df_train = pd.read_csv(training_file,  sep='\t', index_col=0, header=0)
df_test = pd.read_csv(test_file,  sep='\t', index_col=0, header=0)

In [9]:
y_train, y_test = np.loadtxt(training_labs, dtype=np.int), np.loadtxt(test_labs, dtype=np.int)

In [36]:
## Load Coordinates
COORD_FILEPATH = os.path.join('..', 'datasets/coordinates/', 'coordinates_icdf.txt')
coords = pd.read_csv(COORD_FILEPATH, sep='\t', 
                     header=0, index_col=0)
coords = coords[cols].as_matrix().astype(dtype=np.float)

nb_coordinates = coords.shape[0]

### Feature Selection

In [11]:
from sklearn.feature_selection import SelectKBest

In [12]:
kbest = SelectKBest(k=62)
kbest = kbest.fit(df_train, y_train)
index_ranked = kbest.get_support(True)

/home/valerio/anaconda3/lib/python3.5/site-packages/sklearn/feature_selection/univariate_selection.py:113: UserWarning: Features [  6  47  65  76 112 187 235] are constant.
  UserWarning)
/home/valerio/anaconda3/lib/python3.5/site-packages/sklearn/feature_selection/univariate_selection.py:114: RuntimeWarning: invalid value encountered in true_divide
  f = msb / msw


In [13]:
index_ranked

array([  0,   2,  11,  14,  15,  17,  21,  27,  29,  34,  39,  45,  48,
        49,  59,  60,  61,  66,  69,  74,  82,  84,  86,  87,  89,  92,
        96, 101, 102, 103, 105, 109, 113, 114, 115, 122, 123, 125, 127,
       133, 140, 142, 148, 154, 161, 163, 165, 184, 185, 197, 203, 204,
       212, 213, 214, 225, 227, 229, 236, 237, 239, 243])

In [14]:
cols = list()
for i in index_ranked:
    cols.append(df_train.columns.values[i])

In [15]:
df_train_fs = df_train[cols]
df_test_fs = df_test[cols]

### Prepare Data 

In [16]:
X_train = df_train_fs.as_matrix().astype(dtype=np.float)
X_test = df_test_fs.as_matrix().astype(dtype=np.float)

In [17]:
df_train_fs.shape, df_test_fs.shape

((58, 62), (24, 62))

In [18]:
y_train.shape, y_test.shape

((58,), (24,))

In [59]:
X = np.vstack((X_train, X_test))
y = np.hstack((y_train, y_test))
X.shape, y.shape

((82, 62), (82,))

### Visualise Embeddings of Data (ICDF)

Color Code: 

* <span style="color: green;"> Healty Subjects</span>
* <span style="color: red;"> ICDf Subjects</span>


In [60]:
colormap = {1: 'green', 0: 'red'}
colors = [colormap[v] for v in y]

In [63]:
X.shape, X_tsne.shape

((82, 62), (82, 2))

In [61]:
from sklearn.manifold.t_sne import TSNE

In [65]:
tsne = TSNE(n_components=2, n_iter=5000, perplexity=5)
X_tsne = tsne.fit_transform(X)

In [66]:
p = figure(title = "iCDf Data", plot_width=400, plot_height=400)
p.xaxis.axis_label = 'First Component'
p.yaxis.axis_label = 'Second Component'

p.circle(X_tsne[:, 0], X_tsne[:,1],
         color=colors, fill_alpha=0.2, size=10)
show(p)

### Extract PhyloConv Layer Activations

In [34]:
from keras.backend import floatx
from keras.engine import Input, Model
from keras.layers import (Lambda, MaxPooling1D, Flatten,
                          Dropout, Dense, BatchNormalization)


Using TensorFlow backend.


In [35]:
from phcnn.layers import PhyloConv1D, euclidean_distances

Using cuDNN version 5110 on context None
Mapped name None to device cuda0: Tesla K80 (FC0D:00:00.0)


In [37]:
all_coordinates = np.empty((X.shape[0],) + coords.shape, dtype=np.float64)
for i in range(X.shape[0]):
    all_coordinates[i] = coords

In [38]:
all_coordinates = all_coordinates[..., np.newaxis]
X = X[..., np.newaxis]

In [39]:
all_coordinates.shape, X.shape

((82, 306, 62, 1), (82, 62, 1))

In [40]:
nb_features = 62  # current nb of features in the feature step!

# Paramenters for phylo_conv layers
list_filters = [16, 16]
list_neighbours = [4, 4]

# Parameter for output layer
nb_classes = 2

In [41]:
from keras import backend as K

In [10]:
WEIGHT_FILE = os.path.join(os.path.abspath(os.path.curdir), 'icdf_phcnn_model_weights.hdf5')

#### Create PhCNN Network Model

In [42]:
K.clear_session()
data = Input(shape=(nb_features, 1), name="data", dtype=floatx())
coordinates = Input(shape=(nb_coordinates, nb_features, 1),
                    name="coordinates", dtype=floatx())

conv_layer = data
# We remove the padding that we added to work around keras limitations
conv_crd = Lambda(lambda c: c[0], output_shape=lambda s: (s[1:]))(coordinates)

for nb_filters, nb_neighbors in zip(list_filters, list_neighbours):

    if nb_neighbors > nb_features:
        raise Exception("More neighbors than features, "
                        "please use less neighbors or use more features")

    distances = euclidean_distances(conv_crd)
    conv_layer, conv_crd = PhyloConv1D(distances,
                                       nb_neighbors,
                                       nb_filters, activation='selu', trainable=False)([conv_layer, conv_crd])

maxp = MaxPooling1D(pool_size=2, padding="valid")(conv_layer)
flatt = Flatten()(maxp)
fc_1 = Dense(units=128, name='fc_1', activation='sigmoid')(flatt)
dropout = Dropout(0.25, name="dropout_l")(fc_1)
output = Dense(units=nb_classes, kernel_initializer="he_normal",
               activation="softmax", name='new_output')(dropout)

model = Model(inputs=[data, coordinates], outputs=output)
model.load_weights(WEIGHT_FILE, by_name=True)

In [46]:
model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
coordinates (InputLayer)         (None, 306, 62, 1)    0                                            
____________________________________________________________________________________________________
data (InputLayer)                (None, 62, 1)         0                                            
____________________________________________________________________________________________________
lambda_1 (Lambda)                (306, 62, 1)          0           coordinates[0][0]                
____________________________________________________________________________________________________
phylo_conv1d_1 (PhyloConv1D)     [(None, 62, 16), (306 80          data[0][0]                       
                                                                   lambda_1[0][0]          

### Conv Filters

In [44]:
def get_conv_activations(model, layer, Coords, X_batch):
    """Tensorflow function to extract activations from target layer tensor(s).
    This function is supposed to be applied to a Convolutional Layer, so the
    outputs of the function will be the list of all tensors (i.e. cnn filters.)
    
    Parameters:
    -----------
    model: keras.models.Model
        Instance of the target model (i.e. PhCNN network)
    layer: keras.layers.convolutional._Conv
        Instance of the target Convolutional Layer from which 
        extract output tensors
    Coords:  array-like, shape: (batch_size, nb_coords, nb_features, 1)
        Tensor of Coordinates
    X_batch: array-like, shape: (batch_size, nb_features, 1)
        Tensor of batch data
    
    Returns:
    --------
        List of tensor objects.
    """
    
    activations_f = K.function(inputs=[model.layers[0].input, 
                                       model.layers[1].input,
                                       K.learning_phase()], 
                               outputs=[t for t in layer.output])
    activations = activations_f((Coords, X_batch, False))
    return activations

#### Specify Target Network Layer

In [47]:
phyloconv_layer_1 = model.layers[3]

In [48]:
phyloconv_layer_1.name

'phylo_conv1d_1'

#### Get PhyloConv Activations

In [49]:
conv_acts = get_conv_activations(model, phyloconv_layer_1, all_coordinates, X)

In [51]:
conv_acts[0].shape

(82, 62, 16)

In [52]:
X_cnn = conv_acts[0]

#### Plot Embeddings of PhCnn Layer

In [53]:
tsne = TSNE(n_components=2, n_iter=5000, perplexity=5)
for i in range(16):  # nb filters
    try:
        X_cnn_tsne = tsne.fit_transform(X_cnn[:, :, i])
        p = figure(title = "PhyloConv filter {}".format(i+1), 
                   plot_width=400, plot_height=400)
        p.xaxis.axis_label = 'First Component'
        p.yaxis.axis_label = 'Second Component'

        p.circle(X_cnn_tsne[:, 0], X_cnn_tsne[:,1],
                 color=colors, fill_alpha=0.2, size=10)
        show(p)
    except ValueError:
        continue